# Setup

In [2]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig

In [4]:
!nvidia-smi

Mon Oct 21 04:04:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:02:00.0 Off |                  N/A |
|  0%   33C    P8             41W /  420W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Train Model

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
checkpoint = "HuggingFaceM4/idefics-9b"

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=['lm_head', 'embed_tokens']
)

In [8]:
processor = AutoProcessor.from_pretrained(checkpoint, cache_dir='workspace/hf_cache')

preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [41]:
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map='auto', cache_dir='workspace/hf_cache')

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [10]:
model

IdeficsForVisionText2Text(
  (model): IdeficsModel(
    (embed_tokens): IdeficsDecoupledEmbedding(
      num_embeddings=32000, num_additional_embeddings=2, embedding_dim=4096, partially_freeze=False
      (additional_embedding): Embedding(2, 4096)
    )
    (vision_model): IdeficsVisionTransformer(
      (embeddings): IdeficsVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(257, 1280)
      )
      (pre_layrnorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (encoder): IdeficsVisionEncoder(
        (layers): ModuleList(
          (0-31): 32 x IdeficsVisionEncoderLayer(
            (self_attn): IdeficsVisionAttention(
              (k_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
              (v_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
              (q_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
        

In [34]:
# Inference
def do_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors='pt').to(device)
    generated_ids = model.generate(
        **inputs,
        eos_token_id=[eos_token_id],
        bad_words_ids=bad_words_ids,
        max_new_tokens=max_new_tokens,
        early_stopping=True
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
        

In [43]:
url = "https://images.pokemontcg.io/pop6/2_hires.png"

prompts = [
    url,
    "Question: What's on the picture? Answer:",
]

do_inference(model, processor, prompts, max_new_tokens=100)

Question: What's on the picture? Answer: Lucario.


In [20]:
import torchvision.transforms as transforms

In [19]:
##preprocessing
def convert_to_rgb(image):
  if image.mode == "RGB":
    return image

  image_rgba = image.convert("RGBA")
  background = Image.new("RGBA", image_rgba.size, (255,255,255))
  alpha_composite = Image.alpha_composite(background, image_rgba)
  alpha_composite = alpha_composite.convert("RGB")
  return alpha_composite

def ds_transforms(example_batch):
  image_size = processor.image_processor.image_size
  image_mean = processor.image_processor.image_mean
  image_std = processor.image_processor.image_std

  image_transform = transforms.Compose([
      convert_to_rgb,
      transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
      transforms.ToTensor(),
      transforms.Normalize(mean=image_mean, std=image_std)
  ])

  prompts = []
  for i in range(len(example_batch['caption'])):
    caption = example_batch['caption'][i].split(".")[0]
    prompts.append(
        [
            example_batch['image_url'][i],
            f"Question: What's on the picture? Answer: This is {example_batch['name']}. {caption}",
        ],
    )
  inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)
  inputs["labels"] = inputs["input_ids"]
  return inputs

In [21]:
# load and prepare the data for finetuning

ds = load_dataset('TheFusion21/PokemonCards', cache_dir='workspace/hf_cache')
ds = ds['train'].train_test_split(test_size=0.002, seed=42)
train_ds = ds['train']
eval_ds = ds['test']
train_ds.set_transform(ds_transforms)
eval_ds.set_transform(ds_transforms)

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/9.28M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13139 [00:00<?, ? examples/s]

In [23]:
model_name = checkpoint.split('/')[1]
config = LoraConfig(
    r = 16, # if model is small, rank should be higher
    lora_alpha = 32,
    target_modules = ['q_proj', 'k_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none'
)

In [24]:
model = get_peft_model(model, config)

In [25]:
model.print_trainable_parameters()

trainable params: 19,750,912 || all params: 8,949,430,544 || trainable%: 0.2207


In [29]:
training_args = TrainingArguments(
    output_dir = f'{model_name}-PokemonCards',
    learning_rate = 2e-4,
    fp16 = True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy='steps',
    save_strategy='steps',
    eval_steps=25,
    save_steps=50,
    max_steps=50,
    logging_steps=10,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=['labels'],
    load_best_model_at_end=True,
    report_to='none',
    optim='paged_adamw_8bit'
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds
)

max_steps is given, it will override any value given in num_train_epochs


In [31]:
trainer.train()

You may have used the wrong order for inputs. `images` should be passed before `text`. The `images` and `text` inputs will be swapped. This behavior will be deprecated in transformers v4.47.


Step,Training Loss,Validation Loss
25,8.062600,0.803174
50,5.671900,0.726063


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

TrainOutput(global_step=50, training_loss=9.168774032592774, metrics={'train_runtime': 632.9308, 'train_samples_per_second': 1.264, 'train_steps_per_second': 0.079, 'total_flos': 3796581474282816.0, 'train_loss': 9.168774032592774, 'epoch': 0.06101281269066504})

# Inference

In [35]:
url = "https://images.pokemontcg.io/pop6/2_hires.png"

prompts = [
    url,
    "Question: What's on the picture? Answer:",
]

do_inference(model, processor, prompts, max_new_tokens=100)

Question: What's on the picture? Answer: This is ['Lucario', 'Scyther']. A Stage 1 Pokemon Card of type Darkness with the title Lucario and 90 HP of rarity Rare evolved from Pikachu from the set Neo Genesis and the flavor text: It can use its tail to attack enemies from behind, and it can also use its tail to block attacks from the front. It is said that it can use its tail to attack enemies from behind,


# Push To Hub

In [39]:
from huggingface_hub import login
import getpass

# Prompt for token input
token = getpass.getpass("Enter your Hugging Face token: ")

# Log in with the provided token
login(token=token)

Enter your Hugging Face token:  ········


In [40]:
model.push_to_hub(f"{model_name}-PokemonCards", private=False)

adapter_model.safetensors:   0%|          | 0.00/79.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gauravparajuli/idefics-9b-PokemonCards/commit/117f61d6d37deda39add603c683f16fbc6336a78', commit_message='Upload model', commit_description='', oid='117f61d6d37deda39add603c683f16fbc6336a78', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gauravparajuli/idefics-9b-PokemonCards', endpoint='https://huggingface.co', repo_type='model', repo_id='gauravparajuli/idefics-9b-PokemonCards'), pr_revision=None, pr_num=None)